# Problem Set 1 (PS1): A Deeper Dive into Malkiel's Theorem 4
Malkiel proposed five theorems that govern the price of fixed-income debt securities, e.g., Treasury Bills, Notes, and Bonds, as a function of duration, yield, and coupon (interest) rate values:

> [Malkiel, B. G. (1962). Expectations, Bond Prices, and the Term Structure of Interest Rates. The Quarterly Journal of Economics, 76(2), 197–218. https://doi.org/10.2307/1880816](https://www.jstor.org/stable/1880816)

In class, we showed (by simulation) that we could observe the essential stipulation of `Theorem 4`: Price movements resulting from equal absolute increases and decreases in yield are asymmetric; i.e., decreasing yields raise bond prices more than the same increase in yields lowers prices. However, during the discussion of the simulation results, we encountered an interesting question: 
> __Question__: Would you expect the price asymmetry to increase or decrease with the duration of the note or bond?

### Tasks
The objective of this problem set is to explore the price asymmetry question we introduced in the lecture by simulating price changes in different instruments resulting from changes in $\bar{r}$.
* __Setup, Data and Prerequisites__: Load the Treasury auction dataset. We load some data and set up components that we use below. There is nothing to do in the __prerequisite__ section, but check it out if you are curious.
* __Task 1__: Compute prices for a Bill, Note, and Bond to check our implementation. We've selected some Treasury securities from the dataset, but we must build models and compute prices (and other data). In this task, you will create (and populate) models of a Bill, a Note, and a Bond and compare the computed price with the price observed at auction.
* __Task 2__: In class, we showed that notes (and, by extension, bonds) show price change asymmetry. However, we have not explored whether Bills also have this property. In this task, you'll use the treasury bill model we created above and explore the price asymmetry question by simulation.
* __Task 3__: Finally, in class, we showed that coupon securities exhibited price asymmetry, i.e., the absolute percentage change in price was larger for decreases in $\bar{r}\downarrow$ compared to the response when $\bar{r}\uparrow$ increases. However, are longer-maturity instruments more sensitive to this effect, i.e., do they exhibit larger asymmetry? Let's do some calculations to explore this question.

Let's get started!
___

## Setup, Data, and Prerequisites
We set up the computational environment by including the `Include.jl` file, loading any needed resources, such as sample datasets, and setting up any required constants. 

> __Include:__ The [include(...) function](https://docs.julialang.org/en/v1/base/base/#include) is a convenient way to load Julia code from another file. The `Include.jl` file is a Julia file that loads external packages and various functions that we will use in the exercise. It checks for a `Manifest.toml` file; if it finds one, packages are loaded. Otherwise, packages are downloaded and then loaded.

Set up your computational environment by running the code cell below.

In [ ]:
include(joinpath(@__DIR__, "Include.jl")); # this sets up the environment, we'll do this all the time, on everything we do

For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

### Data
We'll explore `T-bill,` `T-note,` and `T-bond` prices from United States Treasury auctions between March 2020 and September 2024, downloaded as a `CSV` file using the [Auction query functionality of TreasuryDirect.gov](https://www.treasurydirect.gov/auctions/auction-query/). We load the `CSV` dataset using the local [`MyTreasuryBillsNotesAndBondsDataSet()` function](src/Files.jl), which returns the auction data as [the `dataset::DataFrame`](https://dataframes.juliadata.org/stable/) variable.

In [2]:
dataset = MyTreasuryBillsNotesAndBondsDataSet()

Row,CUSIP,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price,Investment Rate,High Yield,Interest Payment Frequency,Interest Rate
,String15,String7,String7,String15,String15,String15,Float64,Float64?,Float64?,String15,Float64?
1,912797LS4,Bill,4-Week,09/05/2024,09/10/2024,10/08/2024,99.6049,0.05171,missing,None,missing
2,912797MA2,Bill,8-Week,09/05/2024,09/10/2024,11/05/2024,99.216,0.0515,missing,None,missing
3,912797MW4,Bill,17-Week,09/04/2024,09/10/2024,01/07/2025,98.41,0.04956,missing,None,missing
4,912797KU0,Bill,42-Day,09/03/2024,09/05/2024,10/17/2024,99.4027,0.05222,missing,None,missing
5,912797LF2,Bill,13-Week,09/03/2024,09/05/2024,12/05/2024,98.7437,0.05103,missing,None,missing
6,912797MM6,Bill,26-Week,09/03/2024,09/05/2024,03/06/2025,97.6517,0.04823,missing,None,missing
7,912797MH7,Bill,52-Week,09/03/2024,09/05/2024,09/04/2025,95.8039,0.04345,missing,None,missing
8,912797LK1,Bill,4-Week,08/29/2024,09/03/2024,10/01/2024,99.5979,0.05263,missing,None,missing
9,912797LV7,Bill,8-Week,08/29/2024,09/03/2024,10/29/2024,99.2067,0.05212,missing,None,missing


To help below, let's use [the `filter!(...)` method exported by the DataFrames.jl package](https://dataframes.juliadata.org/stable/lib/functions/#Base.filter) to collect only the data for the bills; we save this to the `bills_dataset::DataFrame` variable.

In [3]:
bills_dataset = filter(Symbol("Security Type") => x-> x == "Bill", dataset)

Row,CUSIP,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price,Investment Rate,High Yield,Interest Payment Frequency,Interest Rate
,String15,String7,String7,String15,String15,String15,Float64,Float64?,Float64?,String15,Float64?
1,912797LS4,Bill,4-Week,09/05/2024,09/10/2024,10/08/2024,99.6049,0.05171,missing,None,missing
2,912797MA2,Bill,8-Week,09/05/2024,09/10/2024,11/05/2024,99.216,0.0515,missing,None,missing
3,912797MW4,Bill,17-Week,09/04/2024,09/10/2024,01/07/2025,98.41,0.04956,missing,None,missing
4,912797KU0,Bill,42-Day,09/03/2024,09/05/2024,10/17/2024,99.4027,0.05222,missing,None,missing
5,912797LF2,Bill,13-Week,09/03/2024,09/05/2024,12/05/2024,98.7437,0.05103,missing,None,missing
6,912797MM6,Bill,26-Week,09/03/2024,09/05/2024,03/06/2025,97.6517,0.04823,missing,None,missing
7,912797MH7,Bill,52-Week,09/03/2024,09/05/2024,09/04/2025,95.8039,0.04345,missing,None,missing
8,912797LK1,Bill,4-Week,08/29/2024,09/03/2024,10/01/2024,99.5979,0.05263,missing,None,missing
9,912797LV7,Bill,8-Week,08/29/2024,09/03/2024,10/29/2024,99.2067,0.05212,missing,None,missing


Similarly, we use [the `filter!(...)` method exported by the DataFrames.jl package](https://dataframes.juliadata.org/stable/lib/functions/#Base.filter) to collect only the data for the notes and bonds; we save this to the `notes_bonds_dataset::DataFrame` variable.

In [4]:
notes_bonds_dataset = filter(Symbol("Security Type") => x-> (x == "Note" || x == "Bond"), dataset)

Row,CUSIP,Security Type,Security Term,Auction Date,Issue Date,Maturity Date,Price,Investment Rate,High Yield,Interest Payment Frequency,Interest Rate
,String15,String7,String7,String15,String15,String15,Float64,Float64?,Float64?,String15,Float64?
1,91282CLJ8,Note,7-Year,08/29/2024,09/03/2024,08/31/2031,99.8775,missing,0.0377,Semi-Annual,0.0375
2,91282CLK5,Note,5-Year,08/28/2024,09/03/2024,08/31/2029,99.9089,missing,0.03645,Semi-Annual,0.03625
3,91282CLH2,Note,2-Year,08/27/2024,09/03/2024,08/31/2026,99.7639,missing,0.03874,Semi-Annual,0.0375
4,912810UD8,Bond,20-Year,08/21/2024,09/03/2024,08/15/2044,99.5248,missing,0.0416,Semi-Annual,0.04125
5,912810UC0,Bond,30-Year,08/08/2024,08/15/2024,08/15/2054,98.9288,missing,0.04314,Semi-Annual,0.0425
6,91282CLF6,Note,10-Year,08/07/2024,08/15/2024,08/15/2034,99.3037,missing,0.0396,Semi-Annual,0.03875
7,91282CLG4,Note,3-Year,08/06/2024,08/15/2024,08/15/2027,99.8314,missing,0.0381,Semi-Annual,0.0375
8,91282CLD1,Note,7-Year,07/25/2024,07/31/2024,07/31/2031,99.7773,missing,0.04162,Semi-Annual,0.04125
9,91282CLA7,Note,2-Year,07/24/2024,07/31/2024,07/31/2026,100.0,missing,missing,Quarterly,missing


Finally, we build an [instance of the `DiscreteCompoundingModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.DiscreteCompoundingModel) and store this discount model in the `discount_model` variable:

In [5]:
discount_model = DiscreteCompoundingModel();

___

## Task 1: Compute prices for a Bill, Note, and Bond to check our implementation
In this task, we will compute the price of a Treasury bill, note, and bond using the treasury model we created in class. 

The teaching team has selected a random bill, note, and bond from the dataset. You will construct the corresponding treasury model instance, compute the price for each of these securities, and compare the calculated and observed prices. 

> __Example:__ Let's compute the price of a `26-Week` T-bill with par value $V_{P}$ = `100 USD` and a value of $\bar{r}$ = `0.04865.` The observed price for this security is $V_{B}$ = `97.6315 USD`. Assume two compounding periods per year.

Create an instance of [the `MyUSTreasuryZeroCouponBondModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryZeroCouponBondModel) using a [custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryZeroCouponBondModel},%20NamedTuple}), assuming the data above. Pass the data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryZeroCouponBondModel},%20NamedTuple}), and use the short-cut syntax to compute the price. Save the updated model in the `zero_coupon_model` variable.

In [6]:
zero_coupon_model = let

    # initialize -
    zero_coupon_model = nothing;

    # TODO: Build a zero_coupon_model instance using the `build(...)` function and the `MyUSTreasuryZeroCouponBondModel` type, 
    # TODO: Pass in the appropriate parameters for the 26-week bill. 
    # TODO: Use the `securityterm(...)` function to convert the string "26-Week" into a `SecurityTerm` type for the `T` parameter.
    # TODO: Finally, pipe the result of the `build(...)` function into the `discount_model` instance to populate the `zero_coupon_model` instance.
    zero_coupon_model = build(MyUSTreasuryZeroCouponBondModel, (
        n = 2, par = 100, 
            T = "26-Week" |> securityterm,
            rate = 0.04865,
    )) |> discount_model;

    zero_coupon_model; # return
end

MyUSTreasuryZeroCouponBondModel(100.0, 0.04865, 0.4986301369863014, 97.63169388203484, 2, Dict(0 => -97.63169388203484, 1 => 100.0), Dict(0 => 1.0, 1 => 1.024257554322746))

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 

> __Test:__ If the computed and observed prices are different, i.e., they have a relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). Otherwise, the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `pass` silently (nothing happens).

So, what do we see?

In [7]:
observed_bill_price = let
    observed_bill_price = 97.6315;
    @assert isapprox(observed_bill_price, zero_coupon_model.price, rtol=1e-4)
    observed_bill_price;
end

97.6315

Next, compute the price of a Treasury Note.

> __Example:__ Compute the price of a `7-Year` T-Note with par value $V_{P}$ = `100 USD`, a value of $\bar{r}$ = `0.04162`, and coupon rate $\bar{c}$ = `0.04125.` The observed sale price for this security at auction was $V_{B}$ = `98.9288 USD`. Assume two compounding periods per year.

Create an [instance of the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}), assuming the data above. Pass the appropriate data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}) and use the short-cut syntax to compute the price. Save the updated model in the `note_coupon_model` variable.

In [8]:
note_coupon_model = let

    # initialize -
    note_coupon_model = nothing;

    # TODO: Build a note_coupon_model instance using the `build(...)` function and the `MyUSTreasuryCouponSecurityModel` type,
    # TODO: Pass in the appropriate parameters for the 7-year note (don't forget to use the `securityterm(...)` function for the `T` parameter).
    # TODO: Finally, pipe the result of the `build(...)` function into the `discount_model` instance to populate the `note_coupon_model` instance.
    note_coupon_model = build(MyUSTreasuryCouponSecurityModel, (
        λ = 2, par = 100.0,
            
        T = "7-Year" |> securityterm,
        rate = 0.04162,
        coupon = 0.04125,
        
    )) |> discount_model;

    note_coupon_model; # return
end;

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 

> __Test:__ If the computed and observed prices are different, i.e., they have a relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). Otherwise, the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `pass` silently (nothing happens).

So, what do we see?

In [9]:
observed_note_price = let
    observed_note_price = 99.7773;
    @assert isapprox(observed_note_price, note_coupon_model.price, rtol=1e-4)
    observed_note_price;
end

99.7773

Next, compute the price of a Treasury Bond.

> __Example:__ Let's compute the price of a `30-Year` T-Bond with par value $V_{P}$ = `100 USD`, a value of $\bar{r}$ = `0.04314`, and coupon rate $\bar{c}$ = `0.0425.` The observed sale price for this security at auction was $V_{B}$ = `98.9288 USD`. Assume two compounding periods per year.

Create an [instance of the `MyUSTreasuryCouponSecurityModel` type](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.MyUSTreasuryCouponSecurityModel) using [a custom `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}), assuming the data above. Pass the appropriate data into [the `build(...)` method](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/fixed/#VLQuantitativeFinancePackage.build-Tuple{Type{MyUSTreasuryCouponSecurityModel},%20NamedTuple}) and use the short-cut syntax to compute the price. Save the updated model in the `bond_coupon_model` variable.

In [10]:
bond_coupon_model = let

    # initialize -
    bond_coupon_model = nothing;

    # TODO: Build a bond_coupon_model instance using the `build(...)` function and the `MyUSTreasuryCouponSecurityModel` type,
    # TODO: Pass in the appropriate parameters for the 30-year bond (don't forget to use the `securityterm(...)` function for the `T` parameter).
    # TODO: Finally, pipe the result of the `build(...)` function into the `discount_model` instance to populate the `bond_coupon_model` instance.
    bond_coupon_model = build(MyUSTreasuryCouponSecurityModel, (
        λ = 2, par = 100.0,
            
        T = "30-Year" |> securityterm,
        rate = 0.04314,
        coupon = 0.0425,
        
    )) |> discount_model;

    bond_coupon_model; # return
end;

Now that we have calculated the price, compare the calculated and observed prices using the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) in combination with [the `isapprox(...)` method](https://docs.julialang.org/en/v1/base/math/#Base.isapprox). 

> __Test:__ If the computed and observed prices are different, i.e., they have a relative difference tolerance of `rtol`$>$`1e-4`, then the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `fail,` resulting in an [AssertionError](https://docs.julialang.org/en/v1/base/base/#Core.AssertionError). Otherwise, the [@assert macro](https://docs.julialang.org/en/v1/base/base/#Base.@assert) will `pass` silently (nothing happens).

So, what do we see?

In [11]:
observed_bond_price = let
    observed_bond_price = 98.9288;
    @assert isapprox(observed_bond_price, bond_coupon_model.price, rtol=1e-4)
    observed_bond_price;
end

98.9288

## Task 2: Do bills show the price asymmetry property? 
In class, we showed that notes (and, by extension, bonds) show price change asymmetry. However, we have not explored whether Bills also have this property. In this task, you'll use the treasury bill model that we created above and explore the price asymmetry question by simulation.

> __Strategy:__ To simulate the asymmetry of price changes following changes in the yield (discount rate), with all other variables held constant, we'll generate a new rate $\bar{r}\leftarrow\beta\cdot\bar{r}$, where $\beta$ is a perturbation value. If $\beta<1$, the perturbed interest rate is _less than_ the nominal rate. If $\beta=1$, the perturbed interest rate _equals_ the nominal rate, and if $\beta>1$, the perturbed interest rate is _greater than_ the nominal rate. We'll then compute the price and the percentage change in price for each perturbed rate.

Let's use the `zero_coupon_model` instance from above for this experiment.

* First, specify the number of perturbation values in the `number_of_samples_task_2` variable. For this simulation, let `number_of_samples_task_2 = 7` (odd, so we capture the nominal case as the center data point).
* Next, specify the lower bound in the `β₁` variable and the upper bound in the `β₂` variable; let `β₁ = 0.8` and `β₂ = 1.2.`
* Finally, compute the perturbation array (stored in the `β::Array{Float64,1}` variable) using the [range function](https://docs.julialang.org/en/v1/base/math/#Base.range) in combination with the [Julia pipe |> operator](https://docs.julialang.org/en/v1/manual/functions/#Function-composition-and-piping), and the [collect function](https://docs.julialang.org/en/v1/base/collections/#Base.collect-Tuple{Type,%20Any}), which converts a range type to a collection, i.e., an array.

Build the perturbation array:

In [1]:
number_of_samples_task_2  = 7;
β₁ = 0.8;
β₂ = 1.2;
β = range(β₁, stop = β₂, length = number_of_samples_task_2) |> collect;

`Finish`: Your job is to complete the implementation of the `Theorem 4` simulation for Bills, which we started below, and analyze the simulation results. We save the simulation results in the `simulation_results_task_2_array::Array{Float64,2}` array and display the results in a table using the `pretty_table(...)` function exported from the [PrettyTables.jl package](https://github.com/ronisbr/PrettyTables.jl).

`Hint`: Check out [the Julia `deepcopy(...)` function](https://docs.julialang.org/en/v1/base/copy/#Base.deepcopy). When you complete your implementation and run the notebook, you should see a table similar (numbers different, but similar structure) to the in-class example. 

What do you see?

In [13]:
simulation_results_task_2_array = let
    
    # initialize -
    VB = observed_bill_price;
    simulation_results_task_2_array = Array{Float64,2}(undef, number_of_samples_task_2, 3);

    # main loop
    for i ∈ eachindex(β)
        β_value = β[i]
       
        ### TODO: Build a deep copy of the zero-coupon model and update its rate. Alternatively, build a new model using the zcm parameters, and update the rate
        model = deepcopy(zero_coupon_model);
        model.rate = β_value*zero_coupon_model.rate
        ### END SOLUTION
        
        # compute: use short-cut syntax and compute the price
        perturbed_price = model |> discount_model |> x-> x.price
        
        # capture: put data in simulation_results_task_2_array
        simulation_results_task_2_array[i,1] = β_value; # col 1: perturbation value
        simulation_results_task_2_array[i,2] = 100*((model.rate - zero_coupon_model.rate)/(zero_coupon_model.rate));    # col 2: percentage change in r̄
        simulation_results_task_2_array[i,3] = 100*((model.price - VB)/(VB)); # col 3: percentage change in the price of the Bill
    end
    simulation_results_task_2_array
end;

In [14]:
# let's make a table to display the results of the simulation for Task 2
pretty_table(simulation_results_task_2_array, header=["β","Δr̄ (%)","ΔPrice (%)"] , tf=tf_simple)

=========== ========== ==============
         β     Δr̄ (%)    ΔPrice (%) 
=========== ========== ==============
       0.8      -20.0      0.476102
  0.866667   -13.3333      0.316965
  0.933333   -6.66667      0.158331
       1.0        0.0   0.000198586
   1.06667    6.66667     -0.157434
   1.13333    13.3333     -0.314571
       1.2       20.0     -0.471212
=========== ========== ==============


#### Does Theorem 4 hold for T-bills?

Based on the table above:
1. Do you observe price asymmetry in T-bills? (Yes/No). Set `does_theorem_4_hold_for_bills` to `true` or `false` based on your observation.
2. Which direction shows larger absolute price changes: rate increases or decreases?
3. In one sentence, explain what this tells us about T-bill price sensitivity to yield changes.

In [15]:
does_theorem_4_hold_for_bills = true; # update this value {true | false}

__Explain your answers here__ to Q2 and Q3 in a few sentences, providing reasoning based on the simulation results. This is a markdown cell, so you can use bullet points, numbered lists, or any other formatting that helps clarify your explanation.

In [16]:
did_I_answer_DQ1_flag = true; # update this value {true | false}

## Task 3: Is price asymmetry a function of maturity?
Finally, in class, we showed that coupon securities exhibited price asymmetry, i.e., the absolute percentage change in price was larger for decreases in $\bar{r}\downarrow$ compared to the response when $\bar{r}\uparrow$ increases. However, are longer-maturity instruments more sensitive to this effect, i.e., do they exhibit larger asymmetry? Let's do some calculations to explore this question.

To start, create a hypothetical `T = 20-year` bond with the same parameters as the `30-year` bond we created above and compute its price. All parameters should be the same __except__ the duration. Save the populated model instance in the `short_duration_bond` variable:

In [17]:
short_duration_bond  = let
    
    # initialize -
    short_duration_bond = nothing;
    
    # TODO: Build a deep copy of the bond_coupon_model
    # TODO: Update its T parameter to "20-Year" (don't forget to use the `securityterm(...)` function).
    # TODO: Pipe the result into the `discount_model` instance to populate the `short_duration_bond` instance.
    short_duration_bond = deepcopy(bond_coupon_model);
    short_duration_bond.T = "20-Year" |> securityterm;
    short_duration_bond |> discount_model;

    short_duration_bond; # return
end

MyUSTreasuryCouponSecurityModel(100.0, 0.04314, 0.0425, 20.0, 2, 99.14825074560238, Dict(5 => 1.909933652139833, 16 => 1.51031954661004, 20 => 1.3867440790618604, 35 => 1.0068722223196547, 12 => 1.6449070649110031, 24 => 1.2732796480913555, 28 => 1.1690989611727236, 8 => 1.791487939269013, 17 => 1.4784298154899222, 30 => 1.1202501563507812…), Dict(5 => 1.112604093665355, 16 => 1.4069870212364195, 20 => 1.5323663768137907, 35 => 2.110496200902611, 12 => 1.2918662976956525, 24 => 1.6689185311218726, 28 => 1.817639114030528, 8 => 1.1861648372955673, 17 => 1.4373357312844892, 30 => 1.8968977490904309…))

`Finish`: Next, finish implementing the code block below. This code computes the percentage change in the short- and long-maturity bond prices and stores the simulation data in the `simulation_results_task_3_array::Array{Float64,2}` array. See the `TODO` block.

In [18]:
simulation_results_task_3_array = let
    
    # initialize -
    simulation_results_task_3_array = Array{Float64,2}(undef, number_of_samples_task_2, 4);
    VB_short = short_duration_bond.price # original
    VB_long = bond_coupon_model.price

    # main loop -
    for i ∈ eachindex(β)
        β_value = β[i]
    
        model_short = deepcopy(short_duration_bond); # create a copy of the short-duration model
        model_long = deepcopy(bond_coupon_model); # create a copy of the original bond model (long duration)
        
        ### TODO: update the rate for the model_short, and model_long instances -
        model_short.rate = β_value*short_duration_bond.rate
        model_long.rate = β_value*bond_coupon_model.rate
        ### END 
        
        # compute: use short-cut syntax and compute the price of the short- and long-duration bonds
        perturbed_price_short = model_short |> discount_model |> x-> x.price
        perturbed_price_long = model_long |> discount_model |> x-> x.price
        
        # capture: put data in simulation_results_task_3_array
        simulation_results_task_3_array[i,1] = β_value; # col 1: store the perturbation value
        simulation_results_task_3_array[i,2] = 100*((model_short.rate - short_duration_bond.rate)/(short_duration_bond.rate));    # col 2: percentage change in r̄
        simulation_results_task_3_array[i,3] = 100*((model_short.price - VB_short)/(VB_short)); # col 3: percentage change in the price of the short bond
        simulation_results_task_3_array[i,4] = 100*((model_long.price - VB_long)/(VB_long)); # col 4: percentage change in the price of the long bond
    end
    
    simulation_results_task_3_array
end;

In [19]:
# let's make a table to display the results of the simulation for Task 3
pretty_table(simulation_results_task_3_array, header=["β","Δr̄ (%)", "ΔPrice-short (%)", "ΔPrice-long (%)"] , tf=tf_simple)

=========== ========== ================== ==================
         β     Δr̄ (%)   ΔPrice-short (%)   ΔPrice-long (%) 
=========== ========== ================== ==================
       0.8      -20.0            12.4282           16.0974
  0.866667   -13.3333            8.07545           10.3548
  0.933333   -6.66667            3.93655           4.99867
       1.0        0.0                0.0               0.0
   1.06667    6.66667           -3.74502          -4.66795
   1.13333    13.3333           -7.30872          -9.02979
       1.2       20.0           -10.7007          -13.1082
=========== ========== ================== ==================


#### Does price asymmetry change with duration?

Based on the table above:
1. Does price asymmetry change with bond duration/maturity? Set `is_price_asymmetry_func_maturity` to `true` or `false` based on your observation.
2. Which bond shows larger absolute price changes for the same yield changes: short-duration (20-year) or long-duration (30-year)?
3. Compare the asymmetry pattern between the two bonds - is the asymmetry effect more pronounced in longer-maturity bonds?
4. In one sentence, explain how maturity affects the price asymmetry phenomenon in coupon-bearing securities.

In [20]:
is_price_asymmetry_func_maturity = true; # update this value {true | false}

__Explain your answers here__ to Q2, Q3, and Q4 in a few sentences, providing reasoning based on the simulation results. This is a markdown cell, so you can use bullet points, numbered lists, or any other formatting that helps clarify your explanation.

In [21]:
did_I_answer_DQ2_flag = true; # update this value {true | false}

## Testing
In the code block below, we compare your answers to those of the teaching team and give you feedback on which items are the same (which presumably means they are correct) and which are different. 

`Unhide` the code block below (if you are curious) about how we implemented the tests and what we are testing.

In [22]:
# let
#     # Set the path -
#     path_to_soln_file = joinpath(_PATH_TO_SOLN,"PS1-TeachingTeam-Solution-CHEME-5660-Fall-2025.jld2")
    
#     # write the file -
#     save(path_to_soln_file, Dict("zero_coupon_model"=>zero_coupon_model, 
#             "note_coupon_model"=>note_coupon_model, "bond_coupon_model" =>bond_coupon_model, 
#             "simulation_results_task_3_array"=>simulation_results_task_3_array, 
#             "simulation_results_task_2_array"=>simulation_results_task_2_array,
#             "is_price_asymmetry_func_maturity"=>is_price_asymmetry_func_maturity,
#             "does_theorem_4_hold_for_bills"=>does_theorem_4_hold_for_bills));
# end

In [23]:
let

    # load teaching team solution data -
    saved_solution_dict = joinpath(_PATH_TO_SOLN,"PS1-TeachingTeam-Solution-CHEME-5660-Fall-2025.jld2") |> load;

    @testset verbose = true "CHEME 5660 problem set 1 test suite" begin

        @testset "include.jl run check" begin
            @test _DID_THIS_INCLUDE_JL_RUN == true
        end

        @testset "zero-coupon model tests" begin
            zero_model_soln = saved_solution_dict["zero_coupon_model"];

            # check the data on the zero model -
            @test isnothing(zero_coupon_model) == false
            @test zero_model_soln.T == zero_coupon_model.T
            @test zero_model_soln.n == zero_coupon_model.n
            @test zero_model_soln.par == zero_coupon_model.par
            @test isapprox(zero_model_soln.rate, zero_coupon_model.rate, rtol=1e-10)  # Use isapprox for rates
            @test isapprox(zero_model_soln.price, zero_coupon_model.price, atol=1e-4)
        end

        @testset "note model tests" begin
            note_model_soln = saved_solution_dict["note_coupon_model"];

            # check the data on the note -
            @test isnothing(note_coupon_model) == false
            @test note_model_soln.T == note_coupon_model.T
            @test note_model_soln.λ == note_coupon_model.λ
            @test note_model_soln.par == note_coupon_model.par
            @test isapprox(note_model_soln.rate, note_coupon_model.rate, rtol=1e-10)  # Use isapprox for rates
            @test isapprox(note_model_soln.coupon, note_coupon_model.coupon, rtol=1e-10)  # Use isapprox for coupon rates
            @test isapprox(note_model_soln.price, note_coupon_model.price, atol=1e-4)
        end

        @testset "bond model tests" begin
            bond_model_soln = saved_solution_dict["bond_coupon_model"];

            # check the data on the bond -
            @test isnothing(bond_coupon_model) == false
            @test bond_model_soln.T == bond_coupon_model.T
            @test bond_model_soln.λ == bond_coupon_model.λ
            @test bond_model_soln.par == bond_coupon_model.par
            @test isapprox(bond_model_soln.rate, bond_coupon_model.rate, rtol=1e-10)  # Use isapprox for rates
            @test isapprox(bond_model_soln.coupon, bond_coupon_model.coupon, rtol=1e-10)  # Use isapprox for coupon rates
            @test isapprox(bond_model_soln.price, bond_coupon_model.price, atol=1e-4)
        end

        @testset "Task 2 tests" begin
            simulation_results_task_2_soln = saved_solution_dict["simulation_results_task_2_array"];
            task_2_flag = saved_solution_dict["does_theorem_4_hold_for_bills"];
            
            @test isapprox(simulation_results_task_2_soln, simulation_results_task_2_array, rtol=1e-6)  # Use isapprox for array comparison
            @test does_theorem_4_hold_for_bills == task_2_flag
            @test did_I_answer_DQ1_flag == true
        end

        @testset "Task 3 tests" begin
            simulation_results_task_3_soln = saved_solution_dict["simulation_results_task_3_array"];
            task_3_flag = saved_solution_dict["is_price_asymmetry_func_maturity"];
            
            @test isapprox(simulation_results_task_3_soln, simulation_results_task_3_array, rtol=1e-6)  # Use isapprox for array comparison
            @test is_price_asymmetry_func_maturity == task_3_flag
            @test did_I_answer_DQ2_flag == true
        end
    end
end;

Test Summary:                       | Pass  Total  Time
CHEME 5660 problem set 1 test suite |   27     27  0.5s
  include.jl run check              |    1      1  0.2s
  zero-coupon model tests           |    6      6  0.0s
  note model tests                  |    7      7  0.0s
  bond model tests                  |    7      7  0.0s
  Task 2 tests                      |    3      3  0.2s
  Task 3 tests                      |    3      3  0.0s
Total  Time
CHEME 5660 problem set 1 test suite |   27     27  0.5s
  include.jl run check              |    1      1  0.2s
  zero-coupon model tests           |    6      6  0.0s
  note model tests                  |    7      7  0.0s
  bond model tests                  |    7      7  0.0s
  Task 2 tests                      |    3      3  0.2s
  Task 3 tests                      |    3      3  0.0s


## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.